In [1]:
import torch
import numpy as np
import torch.nn as nn
import os
import sys
sys.path.append('../')
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torchvision
from voc import *
from coco import *
import torchvision.transforms as transforms
from torchvision.models import resnet152, resnet101, resnet18, resnet34, resnet50
from tqdm import tqdm
import json
import pandas as pd
import matplotlib.pyplot as plt
from config import seed_everything
seed_everything(0)
import timm

# from models import *
from backbones.config import config
import pathlib
from torch import nn, einsum
from einops import rearrange, repeat

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/home/seongha/anaconda3/envs/LTML/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/seongha/anaconda3/envs/LTML/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [2]:
model = timm.create_model(config["vit"], num_classes=80, pretrained=True).to(device)
pre = torch.nn.Sequential(*[model.patch_embed,
model.pos_drop])

print(len(model.blocks))
post = torch.nn.Sequential(*[model.norm,
model.fc_norm,
model.head])

12


/home/seongha/anaconda3/envs/LTML/lib/python3.9/site-packages/torch/cuda/__init__.py:146: UserWarning: 
NVIDIA RTX A5000 with CUDA capability sm_86 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_70.
If you want to use the NVIDIA RTX A5000 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))


In [10]:
inp = torch.randn(1, 3, 224, 224)

def forward_(model, inp):
  inp = pre(inp)
  print(inp.shape)

  int_li = []
  for b in model.blocks:
    inp = b(inp)
    print(inp.shape)
    int_li.append(inp)

  inp = post(inp)
  print(inp.shape)
  return inp, int_li

forward_(model, inp)

torch.Size([1, 196, 768])
torch.Size([1, 196, 768])
torch.Size([1, 196, 768])
torch.Size([1, 196, 768])
torch.Size([1, 196, 768])
torch.Size([1, 196, 768])
torch.Size([1, 196, 768])
torch.Size([1, 196, 768])
torch.Size([1, 196, 768])
torch.Size([1, 196, 768])
torch.Size([1, 196, 768])
torch.Size([1, 196, 768])
torch.Size([1, 196, 768])
torch.Size([1, 196, 80])


(tensor([[[ 0.5977, -0.4091,  1.4682,  ..., -0.4644, -0.0036,  2.7809],
          [ 1.4621, -1.0185,  0.6426,  ..., -0.2896, -1.1233,  1.8893],
          [ 1.5859, -1.5175,  0.5555,  ..., -0.6128, -0.2043,  2.2052],
          ...,
          [ 3.1653,  1.1116, -1.0886,  ..., -0.0912, -1.4020,  1.6420],
          [ 0.5902, -0.3702,  0.4086,  ..., -1.5330, -0.7562,  2.9862],
          [ 1.2250, -1.4316,  0.7843,  ..., -1.5045, -0.4867,  2.2433]]],
        grad_fn=<ViewBackward0>),
 [tensor([[[-1.0536,  0.0774, -0.7863,  ...,  0.5789, -0.1096,  1.9263],
           [-1.1046,  0.0685, -0.7852,  ...,  0.3846,  0.4005,  1.8546],
           [-1.1540, -0.1799, -0.7700,  ...,  0.4842,  0.0243,  1.8204],
           ...,
           [-0.6701,  0.1813, -0.7853,  ...,  0.1094,  0.0572,  2.0724],
           [-1.2050, -0.2024, -0.8298,  ...,  0.2395, -0.0288,  1.1773],
           [-0.9944, -0.0105, -0.9310,  ...,  0.0880,  0.0674,  1.9232]]],
         grad_fn=<AddBackward0>),
  tensor([[[-0.2571,  0.232

In [133]:
print(int_li[0])
lin =torch.nn.Linear(768, 768, bias=False)
rl = torch.nn.ReLU()
out = rl(lin(int_li[0]))
print(out)


tensor([[[-0.6628, -0.2821, -0.8683,  ...,  0.2226, -0.2399,  1.8106],
         [-0.9508,  0.0593, -0.7059,  ...,  0.0527, -0.1204,  2.0101],
         [-0.4276,  0.0181, -0.5251,  ...,  0.1864,  0.2215,  2.6931],
         ...,
         [-1.0209, -0.2023, -1.0002,  ...,  0.2080, -0.0751,  1.4898],
         [-0.7854, -0.1687, -0.6663,  ...,  0.2828, -0.0201,  1.9001],
         [-0.8792,  0.1097, -0.4218,  ...,  0.2727,  0.1519,  1.4047]]],
       grad_fn=<AddBackward0>)
tensor([[[0.1523, 0.1781, 0.0000,  ..., 0.2074, 0.1460, 0.4127],
         [0.0000, 0.0000, 0.1363,  ..., 0.1633, 0.0000, 0.3089],
         [0.0634, 0.0256, 0.0000,  ..., 0.2002, 0.0000, 0.3620],
         ...,
         [0.2519, 0.4729, 0.2573,  ..., 0.2046, 0.1547, 0.4366],
         [0.1115, 0.2103, 0.2342,  ..., 0.0000, 0.0000, 0.1587],
         [0.0582, 0.0910, 0.0000,  ..., 0.1158, 0.0843, 0.5832]]],
       grad_fn=<ReluBackward0>)


In [11]:
dim = 196
inner_dim = 196//1
group_queries = True
group_key_values = True
offset_groups = 4
heads=1
to_q = nn.Conv1d(dim, inner_dim, 1, groups = offset_groups if group_queries else 1, bias = False).to(device)
to_k = nn.Conv1d(dim, inner_dim, 1, groups = offset_groups if group_key_values else 1, bias = False).to(device)
to_v = nn.Conv1d(dim, inner_dim, 1, groups = offset_groups if group_key_values else 1, bias = False).to(device)
to_out = nn.Conv1d(inner_dim, dim, 1).to(device)

In [13]:

group = lambda t: rearrange(t, 'b (g d) n -> (b g) d n', g = offset_groups)
grouped_queries = group(q)
grouped_queries.shape #torch.Size([4, 49, 768])



NameError: name 'q' is not defined

In [14]:
def attention( out, int_li, i=0):
  
  k, v = to_k(int_li[i]), to_v(int_li[i])
  # k.shape, v.shape #(torch.Size([1, 196, 768]), torch.Size([1, 196, 768]))
  q = to_q(out)
  # q.shape #torch.Size([1, 196, 768])

  # split out heads
  q, k, v = map(lambda t: rearrange(t, 'b (h d) n -> b h n d', h = heads), (q, k, v))

  # query / key similarity
  sim = einsum('b h i d, b h j d -> b h i j', q, k)
  # numerical stability

  sim = sim - sim.amax(dim = -1, keepdim = True).detach()

  # attention
  dropout = nn.Dropout(0.0)
  attn = sim.softmax(dim = -1)
  attn = dropout(attn)
  attn.shape

  # aggregate and combine heads

  out = einsum('b h i j, b h j d -> b h i d', attn, v)
  out = rearrange(out, 'b h n d -> b (h d) n')
  out = to_out(out)
  print(out.shape)
  return out

# Datasets

In [15]:
train_dataset = COCO2017('../data/coco', phase='train')
# partial=torch.utils.data.Subset(test_dataset, list(range(100)))
# train_dataset = Voc2007Classification('data/voc', 'trainval', inp_name='data/voc/voc_glove_word2vec.pkl', LT=True)
# test_dataset = Voc2007Classification('data/voc', 'test', inp_name='data/voc/voc_glove_word2vec.pkl')
# train_dataset = COCO2014('data/coco', phase='train', inp_name='data/coco/coco_glove_word2vec.pkl')
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])
train_dataset.transform = transforms.Compose([
                Warp(224),
                transforms.ToTensor(),
                normalize,
            ])

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=50, shuffle=True)


[dataset] Done!
[annotation] Done!
[json] Done!


In [16]:
criterion = torch.nn.MultiLabelSoftMarginLoss()

In [22]:

# model = get_model(i).to(device)
model = model.train()
act_li = np.array([[]])
for i, (input, target) in tqdm(enumerate(train_loader)):
  img, path = input
  target[target == 0] = 1
  target[target == -1] = 0
  feat_Var = torch.autograd.Variable(img).float().to(device)
  
  # output = model(feat_Var, None).detach()
  # output = model(feat_Var)
  output, int_li = forward_(model, feat_Var)
  
  out = attention(output, int_li, i%12)
  out = out.mean(dim=1)
  criterion(out, target)



0it [00:07, ?it/s]


RuntimeError: CUDA error: no kernel image is available for execution on the device
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.